<a href="https://colab.research.google.com/github/jamesphysics/ShorsAlgorithm/blob/main/RSA_Encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RSA encryption using public keys
RSA encyption is underpinned by prime numbers. Let's start with our two "secret" prime numbers (5, 11) and then see how we can use them to encrypt and decrypt a secret message that someone has securely sent to us.

(For more information on this example see: https://thatsmaths.com/2016/08/11/a-toy-example-of-rsa-encryption/ )

In [ ]:
# Secret primes are used as seeds to generate the public key
p, q = 5, 11

# Factorization is difficult so it is safe to share n publicly without revealing
# p and q
n_public = p * q

We require the ability to receive messages from a sender without any prior exchanging of secret information. To achieve this we use our primes to generate, and then publicize, a public key made up of two numbers $(e, n)$.

In the scheme laid out by RSA $e$, the encryption index can be any number less than $n$ so long as it is coprime with Euler's totient function, $\phi =  (p-1)\times(q-1)$

(For more information on the GCD algorithm and it's extended version which shows up later see: https://www.geeksforgeeks.org/euclidean-algorithms-basic-and-extended/ )

In [ ]:
# "Totient function" used to generate encryption keys
phi = (p-1)*(q-1)

# Public Key (e, n) used to encrypt the message
e_public = 7

# Ensure the encryption index is coprime with our totient function by checking 
# The greatest common divisor is 1 (they have no other shared factors)
def gcd(a, b):
  if (a == 0):
    return b
  return gcd(b % a, a)
assert gcd(e_public, phi) == 1

print ('Public key:', n_public, ',', e_public)

Public key: 55 , 7


## Encryption Cipher
To calculate the Cipher message, $C$, the sender follows the formula:

$C = P^e (\text{ mod }n )$

In order to apply this method, the message needs to be broken down into blocks smaller than $n$ which in this case is 55.

Since there are only 26 letters in the alphabet we just convert the message into a string of numbers by assigning each letter a number value based on it's position in the alphabet.

In [ ]:
message = 'secret'
print('Plain-text:', message)

def encrypt(message, e, n):
  encoded = [ord(letter) - 96 for letter in message]
  ciphertext = [(letter ** e) % n for letter in encoded]
  encryptedmessage = ''.join([chr(letter + 96) for letter in ciphertext])
  return str(encryptedmessage)

encryptedmessage = encrypt(message, e_public, n_public)
print('Cipher-text:', encryptedmessage)

Plain-text: secret
Cipher-text: xyqyo


## Decrypting the message
We, the receiver, then apply the decryption scheme:

$D = C^d (\text{ mod }n)$

In [ ]:
def decrypt(encryptedmessage, d, n):
  encryptedmessage = [ord(letter) - 96 for letter in encryptedmessage]
  plaintext = [(letter ** d) % n for letter in encryptedmessage]
  message = ''.join([chr(letter + 96) for letter in plaintext])
  return message

When $ e \times d \equiv 1 \text{ mod } \phi, 0 \leq d \leq n $ 

$D = P$ and so our decrypted and plaintext messages will be the same.

To set this up, the decryption index $d$ is constructed from our secret prime using Euler's extended algorithm

In [ ]:
def gcdExtended(a, b):

    # Base Case
    if a == 0 : 
        return b, 0, 1
            
    gcd, x1, y1 = gcdExtended(b%a, a)
    
    # Update x and y using results of recursive
    # call
    x = y1 - (b//a) * x1
    y = x1
    
    return gcd, x, y

_, _, d_private = gcdExtended(phi, e_public)
d_private %= phi

print ('Private decryption key:', d_private)

Private decryption key: 23


This is our secret key, it is important not to share this with anyone as it can be used to decrypt any messages that are encrypted using our public key.

In [ ]:
print('Encrypted message:', encryptedmessage)

decryptedmessage = decrypt(encryptedmessage, d_private, n_public)
print('Decrypted message:', decryptedmessage)

Encrypted message: xyqyo
Decrypted message: secret


## Vulnerability to Attack
Imagine a hacker is able to steal the encrypted message in transit

In [ ]:
stolenmessage = encryptedmessage

print('Stolen message:', stolenmessage)

Stolen message: xyqyo



The stolen message should be completely meaningless because the hacker does not know $d_{private}$ the private decryption key required to read the message.

The problem is that $n_{public}$ and $e_{public}$ are publicly available. We can use [SymPy](https://www.sympy.org/en/index.html) to factorize $n$, and find $p$ and $q$.

In [ ]:
from sympy.ntheory import factorint

a,b = factorint(n_public)
print ('Factors:', a, ',', b)

Factors: 5 , 11


Once we have these it is simple to calculate $\phi$ and then use the extended Euclidean algorithm to generate $d_{secret}$ the secret decryption key which will expose the secret message.

In [ ]:
phi_stolen = (a-1)*(b-1)
_, _, d_stolen = gcdExtended(phi_stolen, e_public)
d_stolen %= phi_stolen

exposedmessage = decrypt(stolenmessage, d_stolen, n_public)
print('Exposed message:', exposedmessage)

Exposed message: secret


## Safety in numbers
The above vulnerability ignores a crucial fact which is that the usefully available factorint function is actually incredibly slow past a certain point.

$55 \sim 2^6$ and $7 \sim 2^3$ so the encryption key requires only 9 bits to store. Keys used practically will be **at least** 1,024 bits and so much larger chunks of information can be encoded together and securely encrypted.

In this toy example each character will just be predictable substituted for another which is a [simple substitution cipher](https://en.wikipedia.org/wiki/Substitution_cipher) and is very susceptible to attacks such as [frequency analysis](https://en.wikipedia.org/wiki/Frequency_analysis).

The purpose of all this is of course not to use small 9 bit encryption keys that can only encrypt one character, but instead encrypt data using much larger keys.

Google Colab has a handy feature which shows cell execution time.

Trying to factor a 350 bit value took my computer over 2 minutes.

Trying to go above 500 bits gets ridiculous and your computer will start to time out if you try to factor numbers this large.

In [ ]:
exp = 350

integer = 2**exp +1

print(integer)
print(factorint(integer))

2293498615990071511610820895302086940796564989168281123737588839386922876088484808070018553110125686554625
{5: 3, 29: 1, 41: 1, 101: 1, 113: 1, 701: 1, 8101: 1, 268501: 1, 7416361: 1, 47392381: 1, 2430065924693517198550322751963101: 1, 1038213793447841940908293355871461401: 1}


The strong-willed readers may think that if they could just a your computer running long enough then *eventually* the answer would come out.

The reality is that this would take about [300 trillion years](https://www.quintessencelabs.com/blog/breaking-rsa-encryption-update-state-art/), which is orders of magnitude larger than the currently known age of the universe (13.77 billion years).

All of this is to say that, while computers are pretty good at factorizing small numbers, factorizing the keys that protect your bank details (as well as everything else) whilst *theoretically* possible, is in reality functionally impossible.

The fascinating thing is that, according to the same article linked above, a powerful enough quantum computer could do this in as little as 10 seconds!